# Extracción de datos bursátiles de las acciones de las empresas Tesla y GameStop

### Comenzamos importando las librerías necesarias

In [9]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings

### Extraemos los datos bursátiles de Tesla y mostrando las 3 primeras filas

In [ ]:
tesla = yf.Ticker("TSLA")
tesla_data = tesla.history(period="max")
tesla_data.head(3)

$TSLA: possibly delisted; no price data found  (1d 1926-04-02 -> 2025-03-08)


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,


### Mostramos las últimas 3 filas

In [ ]:
tesla_revenue = pd.DataFrame(tesla_data)
tesla_revenue.tail(3)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,


### Extraemos los datos bursátiles de Game Stop y mostrando las 3 primeras filas

In [ ]:
gme = yf.Ticker("GME")
gme_data = gme.history(period="max")
gme_data.head(3)

$GME: possibly delisted; no price data found  (1d 1926-04-02 -> 2025-03-08)


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,


### Mostramos las últimas 3 filas

In [13]:
gme_revenue = pd.DataFrame(gme_data)
gme_revenue.tail()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,


### Trazado del gráfico bursátil para las acciones de Tesla utilizando la función *make_graph*

In [14]:
# Ignorar todas las advertencias
warnings.filterwarnings("ignore", category=FutureWarning)

def make_graph(tesla_data, tesla_revenue, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing=0.3)
    stock_data_specific = tesla_data[tesla_data.index <= '2021-06-14']
    revenue_data_specific = tesla_revenue[tesla_revenue['Date'] <= '2021-04-30']
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data_specific.index), y=stock_data_specific['Close'].astype("float"), name="Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data_specific['Date'], format='%Y-%m-%d'), y=revenue_data_specific['Revenue'].astype("float"), name="Revenue"), row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False,
                      height=900,
                      title=stock,
                      xaxis_rangeslider_visible=True)
    fig.show()

# Obtener el objeto ticker para Tesla
ticker = yf.Ticker("TSLA")
ticker_history = ticker.history(period="max")
tesla_data = pd.DataFrame(ticker_history)

ticker_history.reset_index(inplace=True)

# URL de la página web con los ingresos trimestrales
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm"
html_data = requests.get(url).text
soup = BeautifulSoup(html_data, "html.parser")

# Crear un DataFrame vacío para almacenar los datos de ingresos
tesla_revenue = pd.DataFrame(columns=["Date", "Revenue"])

# Iterar sobre las filas de la tabla para extraer los datos de ingresos
for row in soup.find_all("tr"):
    col = row.find_all("td")
    if len(col) > 1:  # Para asegurarse de que la fila no esté vacía
        date = col[0].text.strip()
        revenue = col[1].text.strip()
        tesla_revenue = pd.concat([tesla_revenue, pd.DataFrame({"Date": [date], "Revenue": [revenue]})], ignore_index=True)

# Asegurarse de que todas las fechas tengan el formato consistente 'YYYY-MM-DD'
tesla_revenue['Date'] = pd.to_datetime(tesla_revenue['Date'], format='%Y-%m-%d', errors='coerce')

# Limpiar la columna Revenue
tesla_revenue["Revenue"] = tesla_revenue['Revenue'].str.replace(r',|\$', "", regex=True)

# Convertir a numérico para evitar futuros problemas
tesla_revenue["Revenue"] = pd.to_numeric(tesla_revenue["Revenue"], errors='coerce')

# Eliminar filas con valores NaN
tesla_revenue.dropna(inplace=True)

# Filtrar filas con valores no vacíos en Revenue
tesla_revenue = tesla_revenue[tesla_revenue['Revenue'] != ""]

# Llamar a la función make_graph con los datos cargados
make_graph(tesla_data, tesla_revenue, 'Tesla')


$TSLA: possibly delisted; no price data found  (1d 1926-04-02 -> 2025-03-08)


### Trazado del gráfico bursátil para las acciones de GameStoputilizando la función *make_graph*

In [6]:
# Ignorar todas las advertencias
warnings.filterwarnings("ignore", category=FutureWarning)

def make_graph(gme_data, gme_revenue, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing=0.3)
    stock_data_specific = gme_data[gme_data.index <= '2021-06-14']
    revenue_data_specific = gme_revenue[gme_revenue['Date'] <= '2021-04-30']
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data_specific.index), y=stock_data_specific['Close'].astype("float"), name="Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data_specific['Date'], format='%Y-%m-%d'), y=revenue_data_specific['Revenue'].astype("float"), name="Revenue"), row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False,
                      height=900,
                      title=stock,
                      xaxis_rangeslider_visible=True)
    fig.show()

#Objeto ticker para 
ticker = yf.Ticker("GME")
ticker_history = ticker.history(period="max")
gme_data = pd.DataFrame(ticker_history)

ticker_history.reset_index(inplace=True)

# URL de la página web con los ingresos trimestrales
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm"
html_data = requests.get(url).text
soup = BeautifulSoup(html_data, "html.parser")

#DataFrame vacío para almacenar los datos de ingresos
gme_revenue = pd.DataFrame(columns=["Date", "Revenue"])

#Iteración sobre las filas de la tabla para extraer los datos de ingresos
for row in soup.find_all("tr"):
    col = row.find_all("td")
    if len(col) > 1:  #Para asegurarse de que no existan filas vacías
        date = col[0].text.strip()
        revenue = col[1].text.strip()
        gme_revenue = pd.concat([gme_revenue, pd.DataFrame({"Date": [date], "Revenue": [revenue]})], ignore_index=True)

#Asegurarse de que todas las fechas tengan el formato consistente 'YYYY-MM-DD'
gme_revenue['Date'] = pd.to_datetime(gme_revenue['Date'], format='%Y-%m-%d', errors='coerce')

#Limpiar la columna Revenue
gme_revenue["Revenue"] = gme_revenue['Revenue'].str.replace(r',|\$', "", regex=True)

#Convertir a numérico para evitar futuros problemas
gme_revenue["Revenue"] = pd.to_numeric(gme_revenue["Revenue"], errors='coerce')

#Eliminar filas con valores NaN
gme_revenue.dropna(inplace=True)

#Filtrar filas con valores no vacíos en Revenue
tesla_revenue = gme_revenue[gme_revenue['Revenue'] != ""]

#Llamar a la función make_graph con los datos cargados
make_graph(gme_data, gme_revenue, 'GameStop')

$GME: possibly delisted; no price data found  (1d 1926-04-02 -> 2025-03-08)
